# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846866811405                   -0.70    4.5         
  2   -7.852317024887       -2.26       -1.53    1.0   39.0ms
  3   -7.852614257076       -3.53       -2.56    1.5   39.0ms
  4   -7.852646040799       -4.50       -2.91    2.8   55.2ms
  5   -7.852646537343       -6.30       -3.23    1.0   40.9ms
  6   -7.852646680738       -6.84       -4.10    1.0   36.5ms
  7   -7.852646686405       -8.25       -5.25    1.5   45.0ms
  8   -7.852646686726       -9.49       -5.63    2.2   51.9ms
  9   -7.852646686730      -11.47       -6.30    1.5   44.0ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846878973433                   -0.70           4.8         
  2   -7.852553299114       -2.25       -1.63   0.80    2.0    348ms
  3   -7.852638412968       -4.07       -2.74   0.80    1.0   33.1ms
  4   -7.852646542913       -5.09       -3.32   0.80    2.0   40.2ms
  5   -7.852646677999       -6.87       -4.23   0.80    1.2   33.9ms
  6   -7.852646686404       -8.08       -4.84   0.80    1.5   36.6ms
  7   -7.852646686720       -9.50       -5.83   0.80    1.8   38.6ms
  8   -7.852646686730      -11.02       -7.01   0.80    1.8   39.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.407417e+01     3.386927e+00
 * time: 0.5814111232757568
     1     1.488437e+00     1.853840e+00
 * time: 0.8805279731750488
     2    -1.643812e+00     2.299497e+00
 * time: 0.918597936630249
     3    -3.840533e+00     1.921135e+00
 * time: 0.9728090763092041
     4    -5.086182e+00     1.807207e+00
 * time: 1.02406907081604
     5    -6.900888e+00     9.495939e-01
 * time: 1.0782248973846436
     6    -7.528961e+00     5.917202e-01
 * time: 1.1296720504760742
     7    -7.721367e+00     5.287772e-01
 * time: 1.1659860610961914
     8    -7.783771e+00     1.886031e-01
 * time: 1.2030620574951172
     9    -7.816942e+00     1.319810e-01
 * time: 1.238821029663086
    10    -7.835412e+00     6.330321e-02
 * time: 1.2743759155273438
    11    -7.843874e+00     8.631418e-02
 * time: 1.3109641075134277
    12    -7.848761e+00     3.110621e-02
 * time: 1.3506519794464111
    13    -7.851455e+00     3.107820e-02
 * time: 1.38805890083312

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846835427879                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645883009                   -1.64         
  2   -7.852646686730       -6.09       -3.71    3.36s
  3   -7.852646686730      -13.26       -7.23    286ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.64794762986768e-7
|ρ_newton - ρ_scfv| = 7.464399839173464e-8
|ρ_newton - ρ_dm|   = 4.789679314575421e-10
